<h1>Project-4</h1> 
<h2><u>Aim : Prediction Of The Next Word</u></h2>
<h3 style="color:DodgerBlue;"><u> D A T A S C I E N C E &nbsp I N T E R N S H I P</u></h3> 
<h4>Deep Learning model use for the Prediction onf the data <u> Tensorflow .</u></h4>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding , LSTM , Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [2]:
f1 = open('Story.txt' , 'r' , encoding= "utf8")

In [3]:
lines = []
for i in f1:
    lines.append(i)


In [4]:
whole_string = ""
for i in lines:
      whole_string = " ".join(lines)
        
final_string = whole_string.replace("\n" , '').replace('*' , "").replace('=', '').replace('"' , ' ').replace('_' , '').replace("-", "").replace('"' , "")
final_string[:346]

'The Project Gutenberg eBook of Golden rules of medical evidence, by Stanley B. Atkinson  This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or reuse it under the terms of the Project Gutenberg License included wi'

In [5]:
data = final_string.split()
data  = ' '.join(data)
data[:67]

'The Project Gutenberg eBook of Golden rules of medical evidence, by'

In [6]:
len(data)

60275

In [7]:
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts([data])

# save tokenizer for the predict function
pickle.dump(tokenizer_obj , open('token.pkl' , 'wb'))

sequence_data = tokenizer_obj.texts_to_sequences([data])[0]
len(sequence_data)    # Repeted Word Deleted 

9993

In [8]:
vocab_size = len(tokenizer_obj.word_index )
vocab_size +=1

In [9]:
combination_data = []

for i in range(3 , len(sequence_data)):
    words = sequence_data[i-3:i+1]
    combination_data.append(words)
    
combination_array = np.array(combination_data)

In [10]:
len(combination_array)
combination_array

array([[   1,   12,   36,  106],
       [  12,   36,  106,    2],
       [  36,  106,    2,   56],
       ...,
       [2340,    3, 2341,  140],
       [   3, 2341,  140,  591],
       [2341,  140,  591,  214]])

In [11]:
x = [] 
y = []

for i in combination_array:
    x.append(i[0:3])
    y.append(i[3])

X_array = np.array(x)
Y_array = np.array(y)

In [12]:
print("Data : ",X_array[:10] )
print("Response : ",Y_array[:10])
X_array.shape

Data :  [[  1  12  36]
 [ 12  36 106]
 [ 36 106   2]
 [106   2  56]
 [  2  56  45]
 [ 56  45   2]
 [ 45   2  17]
 [  2  17  20]
 [ 17  20  11]
 [ 20  11 314]]
Response :  [106   2  56  45   2  17  20  11 314  70]


(9990, 3)

In [13]:
# convertion into byte matrix

In [14]:
matrix = to_categorical(Y_array ,  )
matrix.shape

(9990, 2342)

In [15]:
# Creating Model

In [16]:
model = Sequential()
model.add(Embedding(vocab_size , 10 , input_length=3))
model.add(LSTM(1000 , return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000 , activation="relu"))
model.add(Dense(vocab_size , activation="softmax"))

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             23420     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 2342)              2344342   
                                                                 
Total params: 15,416,762
Trainable params: 15,416,762
Non-trainable params: 0
_________________________________________________________________


In [18]:
# plot the model visual look out of the model 

In [19]:
from tensorflow import keras
from keras.utils.vis_utils import plot_model

In [20]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5" , monitor= 'loss' , verbose= 1  , save_best_only = True)
model.compile(loss="categorical_crossentropy" , optimizer=Adam(learning_rate=0.01))
model.fit(X_array , matrix , epochs=10, batch_size = 64 ,callbacks=[checkpoint])

Epoch 1/10
157/157 [==============================] - ETA: 0s - loss: 7.1001
Epoch 1: loss improved from inf to 7.10007, saving model to next_words.h5
157/157 [==============================] - 104s 627ms/step - loss: 7.1001
Epoch 2/10
157/157 [==============================] - ETA: 0s - loss: 6.3949
Epoch 2: loss improved from 7.10007 to 6.39494, saving model to next_words.h5
157/157 [==============================] - 93s 594ms/step - loss: 6.3949
Epoch 3/10
157/157 [==============================] - ETA: 0s - loss: 6.3190
Epoch 3: loss improved from 6.39494 to 6.31905, saving model to next_words.h5
157/157 [==============================] - 94s 595ms/step - loss: 6.3190
Epoch 4/10
157/157 [==============================] - ETA: 0s - loss: 6.2309
Epoch 4: loss improved from 6.31905 to 6.23087, saving model to next_words.h5
157/157 [==============================] - 95s 603ms/step - loss: 6.2309
Epoch 5/10
157/157 [==============================] - ETA: 0s - loss: 6.1549
Epoch 5: loss 

In [36]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl' , 'rb'))

def Prediction_of_the_next_word(model , tokenizer , text):
    seq = tokenizer_obj.texts_to_sequences([text])
    seq = np.array(seq)
    pred = np.argmax(model.predict(seq))
    predicted_word =  ""
    
    for key, value in tokenizer.word_index.items():
        if value == pred:
            predicted_word = key
            break
        
    print(predicted_word)
    return predicted_word
    

In [46]:

text = input("Enter String : ")

text = text.split(" ")
text = text[-3:]
print(text)
Prediction_of_the_next_word(model , tokenizer, text)
            

Enter String :   Neuromuscular action and sudden death,


['and', 'sudden', 'death,']
1/1 [==============================] - 0s 28ms/step
the


'the'